<a href="https://colab.research.google.com/github/weathon/3d2smile/blob/main/thesis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget file.weasoft.com/images.zip
!wget file.weasoft.com/summary.csv
!git clone https://github.com/suanfaxiaohuo/SwinOCSR.git
!pip install focal_loss_torch wandb
!pip3 install deepsmiles yacs tqdm
!wget file.weasoft.com/model.py -O model.py
!wget file.weasoft.com/reverse.map

In [ ]:
!unzip images.zip

In [28]:
!mkdir -p images
!mv *.png images
!mv *.webp images

In [5]:
from model import *

--2024-03-18 23:44:17--  http://file.weasoft.com/model.py
Resolving file.weasoft.com (file.weasoft.com)... 149.28.13.194
Connecting to file.weasoft.com (file.weasoft.com)|149.28.13.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3535 (3.5K) [text/x-python]
Saving to: ‘model.py’

model.py            100%[===================>]   3.45K  --.-KB/s    in 0s      

2024-03-18 23:44:17 (48.2 MB/s) - ‘model.py’ saved [3535/3535]



In [8]:
import torch, torchvision
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
mod = torch.load("drive/MyDrive/final.with.test.7.pt", map_location=device)

In [41]:
import numpy as np
import os
import random
class SMILESGenerator(torch.nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder

  def predict_next(self, i, out, beam, r):
    reward = torch.zeros(self.top_n_p.shape)
    if i>1:
      for j in range(beam):
        if self.top_n[j][-1]==78:
          out[j]=torch.zeros(79)
          out[j][78]=1
          reward[j]=(0.)
        else:
          reward[j]=(r)
    self.top_n_p+=reward.to(device)
    tmp_n = self.top_n_p.repeat((79,1)).T+torch.log(out)
    x, y = top_k_2d(tmp_n, beam)
    new_top_n = []
    new_top_n_p = []
    for j in range(beam):
      new_top_n.append(torch.cat((self.top_n[x[j]], y[j].cpu().unsqueeze(-1))))
      new_top_n_p += [tmp_n[x[j], y[j]]]
    self.top_n = new_top_n
    self.top_n_p = torch.tensor(new_top_n_p).to(device)


  def forward(self, images, text_in_, max_len, beam, r=0.05): #just changed beam=1 and it runs so beam cannot be random number???  yeah it has to been factor of 676, which is 2 2 13 13 but original paper did not use beam search
    with torch.no_grad():
      image_feature = []
      for i in images:
        mem = self.decoder.encoder(self.decoder.encoder_dim(self.encoder(i)))
        mem = mem.repeat_interleave(beam, dim=0)
        image_feature.append(mem)


      self.top_n = torch.tensor(text_in_)
      self.top_n_p = torch.tensor([0.]).to(device)
      for i in range(max_len):
        padded_text , l = pad_pack(self.top_n)
        padded_text = padded_text.to(device)
        out = []
        for i in range(len(images)):
          out.append(torch.nn.functional.softmax(self.decoder.decoder(padded_text, image_feature[i], x_mask=triangle_mask(l).to(device))[:,-1,:].squeeze(dim=1), dim=1))
        out = torch.mean(torch.stack(out), dim=0)
        self.predict_next(i, out, beam, r)
        count = 0
        for i in self.top_n:
          if 78 in i:
            count+=1
        if count==beam:
          return self.top_n, self.top_n_p
      return self.top_n, self.top_n_p

val_cids = [286]#, 6587, 6562, 11203]
val_names = [i for i in os.listdir("./images") if int(i.split("_")[0]) in val_cids]


mod = mod.eval()
gen = SMILESGenerator(mod.encoder, mod.decoder)
img1 = torch.tensor(np.array(Image.open("images/"+random.choice(val_names)).convert("RGB").resize((400,400)))).unsqueeze(0).permute(0,3,1,2).to(device).to(torch.float32)
gen = gen.train(False) #forgot gen=
res = gen.forward([img1], [[77]], 100, 10, 0)
print()
for i in res[0]:
  print(converter.decode("".join([reversed_word_map_[i] for i in i.numpy()])))
print(torch.exp(res[1]))



<start>C(C(C(=O)[O-])O)ON<end>
<start>C(C(C(=O)O)O)ON<end><end><end><end>
<start>C(C(=O)O)ON<end><end><end><end><end><end><end><end>
<start>C(C(C(=O)O)O)ON<end><end><end>
<start>C(C(C(=O)[O-])N)ON<end>
<start>C(C(C(=O)O)ON)ON<end><end>
<start>C(C(C(=O)O)O)ON<end><end(>)N
<start>C(C(=O)O)ON<end><e(nd>)O<end><end>
<start>C(C(C(=O)O)O)ON<end><end>
<start>C(C(=O)O)ON<end><end><end><end><end><end><end>
tensor([0.1834, 0.1043, 0.0456, 0.0261, 0.0201, 0.0196, 0.0129, 0.0104, 0.0088,
        0.0086])


In [14]:
t = 1
import deepsmiles
from PIL import Image
import json


converter = deepsmiles.Converter(rings=True, branches=True)
def triangle_mask(size):
    mask = 1- np.triu(np.ones((1, size, size)),k=1).astype('uint8')
    mask = torch.autograd.Variable(torch.from_numpy(mask))
    return mask

def top_k_2d(m, k):
  values, indices = torch.topk(m.flatten(), k)
  return indices//m.shape[1], indices%m.shape[1]

def pad_pack(sequences):
    maxlen = max(map(len, sequences))
    batch = torch.LongTensor(len(sequences),maxlen).fill_(0)
    for i,x in enumerate(sequences):
        batch[i,:len(x)] = torch.LongTensor(x)
    return batch, maxlen

reversed_word_map = {}

with open("reverse.map","r") as f:
  reversed_word_map = json.loads(f.read())
reversed_word_map_={}

for i in reversed_word_map.keys():
  reversed_word_map_[int(i)] = reversed_word_map[i]
